# The Amplitude Estimation Benchmark Test Case

In the **01_AmplitudeEstimationKernel.ipynb** notebook the **AE kernel** was explained. In this notebook, the associated benchmark test case (**BTC**) is presented. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sys
sys.path.append("../../")

## 0. The QPU

To simulate the quantum circuits generated by the functions presented in this notebook a configured myQLM (or QLM) **Quantum Process Unit (QPU)** is mandatory. The **QPU** can execute an ideal simulation or can simulate the quantum circuits under a noisy hardware model (noisy simulation). To easily deal with these 2 kinds of simulations the  *select_qpu* function from **QQuantLib.qpu.select_qpu** was developed. The input of this function is a Python dictionary that allows to the user configure easily a **QPU**.

In the present notebook, only ideal simulation is used. Please refer to the **QQuantLib/qpu/NoisyModels.ipynb** notebook for configuring noisy models and the corresponding noisy **QPU**s.


In [ ]:
from QQuantLib.qpu.select_qpu import select_qpu

The minimum Python dictionary for configuring an ideal **QPU** is presented in the following cell. In this case, the user only has to provide a value to the *qpu_type* key. Depending on the type of simulator desired the following strings should be provided:

* *qlmass_linalg*: to use the **LinAlg Quantum Learning Machine (QLM)** algebra simulator. In this case, the computation will be sent to the **QLM** by using the  Qaptiva QLM as a Service.
* *qlmass_mps*: to use **MPS QLM** simulator. In this case, the computation will be sent to the **QLM** by using the  Qaptiva QLM as a Service.
* *python*: to use the PyLinalg algebra simulator.
* *c*: to use the CLinalg alegbra simulator.
* *linalg*: to use the **LinAlg QLM**. In this case, the user should be inside a **EVIDEN QLM**
* *mps*: to use the **MPS QLM** simulator. In this case, the user should be inside a **EVIDEN QLM**

In [ ]:
# List with the strings taht should be provided for an ideal QPU
ideal_qpus = ["c", "python", "linalg", "mps", "qlmass_linalg", "qlmass_mps"]

qpu_config = {
    #the following strings can be used:
    #
    "qpu_type": ideal_qpus[0], 
    # The following keys are used for configuring noisy simulations
    "t_gate_1qb" : None,
    "t_gate_2qbs" : None,
    "t_readout": None,
    "depol_channel" : {
        "active": False,
        "error_gate_1qb" : None,
        "error_gate_2qbs" : None
    },
    "idle" : {
        "amplitude_damping": False,
        "dephasing_channel": False,
        "t1" : None,
        "t2" : None
    },
    "meas": {
        "active":False,
        "readout_error": None
    }
}

qpu = select_qpu(qpu_config)

## 1.  Description of the problem

The benchmark problem is the computation of the integral of a function $f(x)$ in a closed interval $[a,b] \subset \mathbf{R}$:

$$I = \int_a^bf(x)dx$$

In particular we propose to use $f(x) = \sin x$, whose integral is very well known:

$$I = \int_a^{b}\sin(x)dx = -\cos x |_a^b = \cos(a)-\cos(b)$$

Additionally, 2 different integration intervals will be used:

1. $[0, \frac{3\pi}{8}]$ : $I = \int_0^{\frac{3\pi}{8}}\sin(x)dx = 0.6173165676349102$
2. $[\pi, \frac{5\pi}{4}]$ : $I = \int_{\pi}^{\frac{5\pi}{4}}\sin(x)dx = -0.2928932188134523$


In [ ]:
def sin_integral(a,b):
    return np.cos(a)-np.cos(b)

In [ ]:
start = [0.0, np.pi]
end = [3.0*np.pi/8.0, 5.0*np.pi/4.0]

#### $1^{st}$  Domain: $[0, \frac{3\pi}{8}]$ 

In this domain, the function is strictly positive defined.

In [ ]:
a = start[0]
b = end[0]
domain = np.linspace(a,b, 100)
plt.plot(domain, np.sin(domain))
#plt.show()

In [ ]:
print('Integral in first domain: {}'.format(sin_integral(a,b)))

#### $2^{nd}$ Domain: $[\pi, \frac{5\pi}{4}]$

In this domain, the function is not strictly positively defined and the integral is negative!

In [ ]:
a = start[1]
b = end[1]
domain = np.linspace(a,b, 100)
plt.plot(domain, np.sin(domain))

In [ ]:
print('Integral in second domain: {}'.format(sin_integral(a,b)))

## 2. BTC Workflow

Now we present the complete Workflow for the **BTC** for the **AE kernel**:
1. Domain discretization
2. Function discretization
3. Array normalisation
4. Encoding function in a quantum state
5. Amplitude Estimation algorithm.

### 2.1. Domain Discretization

The first thing to do for computing the integral in a computer is the discretization of the domain. In the benchmark we always discretize the domain in $2^n$ intervals, with $n \in \mathbf{N}$:

$$\{[x_0, x_1], [x_1, x_2], ..., [x_{2^n-1}, x_{2^n}]\}$$ 

Where

1. $x_{i+1} < x_{i}$
2. $a = x_0$
3. $b = x_{2^n}$

In [ ]:
#First integration domain
a = start[0]
b = end[0]
#For fix the number of discretization intervals
n=4
domain_x = np.linspace(a, b, 2**n+1)

### 2.2. Function discretization

Now using the domain discretization we need to discretise $f(x)$. In our procedure following arrays should be constructed:

1. $\Delta x_i = x_{i+1} - x_{i} = \frac{b-a}{2^n}$
2. $f_{x_i} = \frac{f(x_{i+1}) + f(x_{i})}{2}$
3. $f_{x_i} \Delta x_i = f_{x_i} \frac{b-a}{2^n}$

Using these computed arrays the desired integral can be approximated by the Riemann sum:

$$S_{[a,b]} = \sum_{i=0}^{2^n-1} f_{x_i} \Delta x_i$$

When $\Delta x_i \rightarrow 0$ then $I =\int_a^{b}\sin(x)dx \approx S_{[a,b]}$.

Using $\Delta x_i  = \frac{b-a}{2^n}$ then we can write down:


$$S_{[a,b]} = \sum_{i=0}^{2^n-1} f_{x_i} \frac{b-a}{2^n} = \frac{b-a}{2^n} \sum_{i=0}^{2^n-1} f_{x_i}$$

In [ ]:
#The selected fucntion
f = np.sin

#Discretization of the selected function
f_x = []
x_ = []
for i in range(1, len(domain_x)):
    step_f = (f(domain_x[i]) + f(domain_x[i-1]))/2.0
    #print(i)
    f_x.append(step_f)
    x_.append((domain_x[i] + domain_x[i-1])/2.0)
f_x = np.array(f_x)
x_ = np.array(x_)


In [ ]:
plt.plot(domain_x, f(domain_x), '-o')
plt.plot(x_, f_x, 'o')
plt.legend(['original', 'half'])

In [ ]:
Riemann = (np.sum(f_x)*(b-a))/2**n
print("Riemann sum integral: {}".format(Riemann))
print('Exact Integral in first domain: {}'.format(sin_integral(a,b)))

### 2.3. Array Normalisation

The idea is to encode the $2^n$ discretized array $f_{x_i}$ in a $n+1$ qubit circuit. Before doing that we are going to normalise the array in the following way:


$$f^{norm}_{x_i} = \frac{f_{x_i}}{\max(|f_{x_i}|)}$$

If $\max{|f_{x_i}|} \leq 1$ then this step can be omitted.

Now the computed integral will be as shown in $(1)$

\begin{equation}
S_{[a,b]} = \frac{b-a}{2^n} \sum_{i=0}^{2^n-1} f_{x_i} = \frac{b-a}{2^n} \sum_{i=0}^{2^n-1}  \max(|f_{x_i}|) f^{norm}_{x_i} =\frac{\max(|f_{x_i}|)(b-a)}{2^n} \sum_{i=0}^{2^n-1} f^{norm}_{x_i} \tag{1}
\end{equation}

In [ ]:
normalization = np.max(np.abs(f_x))
print("Normalization constant: {}".format(normalization))
#normalization = 1.0
f_norm_x = f_x/normalization
Riemann = normalization * np.sum(f_norm_x)*(b-a)/2**n
#Now we need to be aware of the normalization constant when computing Rieman sum
print("Riemman sum integral: {}".format(Riemann))
print('Exact Integral in first domain: {}'.format(sin_integral(a,b)))

### 2.4. Encoding function in a quantum circuit.

The next step is to codify $f^{norm}_{x_i}$ array in a quantum circuit. The following procedure must be used:

1. Initialize a quantum register with at least $n+1$ qubits, where $n$ must be equal to the $n$ used to define the $2^n$ discretization intervals $|0\rangle\otimes|0\rangle_n $
2. Apply the uniform distribution over the first $n$ qubits: $$\big(I \otimes H^{\otimes n}\big)\big(|0\rangle \otimes|0\rangle_{n}\big) = |0\rangle \otimes H^{\otimes n}|0\rangle_{n}=
\frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}|0\rangle \otimes|i\rangle_{n} \tag{2}$$
3. Creates an operator $\mathbf{U}_f$ for encoding the $f^{norm}_{x_i}$. This operator acts in the following way (see 2.4.1): $$\mathbf{U}_f \left(|0\rangle \otimes |i\rangle_n\right) =  \big(f^{norm}_{x_i}|0\rangle +  \beta_i |1\rangle \big) \otimes |i\rangle_n  \tag{3}$$ In the equation $(3)$ the coefficient of $|1\rangle$ it is not important for us the only important coefficient it is the $|0\rangle$ one.
4. Apply the $\mathbf{U}_f$ operator over $n+1$ qubits: $$\mathbf{U}_f\left(I\otimes H^{\otimes n}\right)|0\rangle\otimes|0\rangle_{n} \tag{4}$$
5. Applying equation $(2)$ and  $(3)$ on $(4)$: $$\mathbf{U}_f\left(I\otimes H^{\otimes n} \right)|0\rangle\otimes|0\rangle_{n} = \mathbf{U}_f \left(\frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1} |0\rangle\otimes|i\rangle_{n}\right) = \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1} \mathbf{U}_f \left(|0\rangle\otimes|i\rangle_{n}\right) = \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1} |i\rangle_{n} \otimes \left(f^{norm}_{x_i}|0\rangle + \beta_i|1\rangle \right) \tag{5}$$
6. Finally the uniform distribution will be applied over the first $n$ qubits again: $$|\Psi \rangle = \left(I\otimes H^{\otimes n} \right)\mathbf{U}_f\left(I\otimes H^{\otimes n} \right)|0\rangle\otimes|0\rangle_{n} \tag{6}$$
7. So applying $(5)$ on $(6)$: $$|\Psi \rangle = \left(I\otimes H^{\otimes n} \right)\mathbf{U}_f\left(I\otimes H^{\otimes n} \right)|0\rangle\otimes|0\rangle_{n} = \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1} H^{\otimes n}|i\rangle_{n} \otimes \left(f^{norm}_{x_i}|0\rangle + \beta_i|1\rangle \right) \tag{7}$$
8. We are interested only in $|0\rangle \otimes |i\rangle_{n}$ so we don't need to take into account other terms, so $(7)$ can be expressed as $$|\Psi \rangle = \left(I\otimes H^{\otimes n} \right)\mathbf{U}_f\left(I\otimes H^{\otimes n} \right)|0\rangle\otimes|0\rangle_{n} = \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1} f^{norm}_{x_i}|0\rangle \otimes H^{\otimes n}|i\rangle_{n}   + \cdots \tag{8}$$
9. It is known that: $$H^{\otimes n} = \frac{1}{\sqrt{2^n}} \sum_{j=0}^{2^n}\sum_{k=0}^{2^n} (-1)^{jk} |j\rangle_n  {}_{n} \langle k|$$.
10. So:$$H^{\otimes n} |i\rangle_n = \frac{1}{\sqrt{2^n}} \sum_{j=0}^{2^n}\sum_{k=0}^{2^n} (-1)^{jk} |j\rangle_n  {}_{n} \langle k|i\rangle_n = \frac{1}{\sqrt{2^n}} \sum_{j=0}^{2^n} (-1)^{ji} |j\rangle = \frac{1}{\sqrt{2^n}} |0\rangle_n + \frac{1}{\sqrt{2^n}} \sum_{j=1}^{2^n} (-1)^{ji} |j\rangle \tag{9}$$
11. Finally applying $(9)$ in $(8)$ and taking only into account the $|0\rangle \otimes |0\rangle_{n}$   $$|\Psi \rangle = \left(I\otimes H^{\otimes n} \right)\mathbf{U}_f\left(I\otimes H^{\otimes n} \right)|0\rangle\otimes|0\rangle_{n} = \frac{1}{2^n} \sum_{i=0}^{2^{n}-1} f^{norm}_{x_i} |0\rangle\otimes|0\rangle_{n} + \cdots \tag{10}$$

Using this procedure the two mandatory operators $\mathbf{A}^I(f_{x_i})$ (one for each integration interval) can be created:
$$\mathbf{A}^I(f_{x_i}) = \left(\mathbb{I}\otimes H^{\otimes n} \right)\mathbf{U}^I_f\left(\mathbb{I}\otimes H^{\otimes n} \right)$$

Using $(10)$ the behaviour of these operators can be summarised as:

$$|\Psi \rangle = \mathbf{A}^I(f_{x_i}) |0\rangle\otimes|0\rangle_{n}= \frac{1}{2^n} \sum_{i=0}^{2^{n}-1} f^{norm}_{x_i} |0\rangle\otimes|0\rangle_{n} + \cdots \tag{11}$$

$(11)$ can be compared with the equation of the **AE kernel** (see notebook *01_AmplitudeEstimationKernel.ipynb*):

$$|\Psi\rangle= \mathbf{A}|0\rangle_n  = \sqrt{a} |\Psi_0\rangle  + \sqrt{1-a}|\Psi_1\rangle \tag{12}$$

Where $|\Psi_0\rangle = |0\rangle\otimes|0\rangle_{n}$ and $$\sqrt{a} = \frac{1}{2^n} \sum_{i=0}^{2^{n}-1} f^{norm}_{x_i}$$

Now the Riemann sum approximation of the desired integral can be computed by measuring the probability of obtaining the state $|\Psi_0\rangle = |0\rangle \otimes |0\rangle_n$ as shown in $(12)$:
$$\mathbf{P}[|\Psi_0\rangle] = \left| \; \langle \Psi_0\ |\Psi\rangle \; \right|^2 = \left| \; \langle \Psi_0\ | \frac{1}{2^n} \sum_{i=0}^{2^{n}-1} f^{norm}_{x_i} |\Psi_0\rangle\; \right|^2 = \left| \frac{1}{2^n} \sum_{i=0}^{2^{n}-1} f^{norm}_{x_i} \right|^2 = \tilde{a} \tag{13}$$

The $\thicksim$ in $\tilde{a}$ indicates that the amplitude was obtained using a quantum measurement. Now plugging $(13)$ into the definition of the integral $(1)$:

$$\tilde{S}_{[a,b]}= \frac{\max(f_{x_i}) \left(b-a\right)}{2^n}\left(2^n\sqrt{\mathbf{P}[|\Psi_0\rangle]}\right) \tag{14}$$

The $\thicksim$ in $\tilde{S}_{[a,b]}$ indicates that the integral was obtained using a measurement meanwhile the $S_{[a,b]}$ is for pure Riemann sum calculation as shown

The encoding of the array $f^{norm}_{x_i}$ in a unitary operator $\mathbf{A}^I(f_{x_i})$ is done transparently by python class *Encoding* from the *QQuantLib/DL/encoding_protocols* module.

This class creates the operator $\mathbf{A}^I(f_{x_i})$ under the *oracle* property of the *Encoding* class. 

For instantiating the class the following arguments should be provided:

* array_function: the array function $f^{norm}_{x_i}$.
* array_probability: should be fixed to None
* encoding: should be fixed to 2 (the class implement until 3 different encoding protocols but for the **BTC** the it should be fixed to 2)

The *Encoding* class has the following useful attributes:
* oracle: **myqlm** gate with the implementation of the operator $\mathbf{A}^I(f_{x_i})$
* target: list with the binary representation of the *good* state of the $\mathbf{A}^I(f_{x_i})$ operator(this is the $\ket{\Psi_0}$ in $(12)$). So in our case it will be: $|\Psi_0\rangle = |0\rangle \otimes |0\rangle_n$
* index: list with the qubits affected by the unitary operator $\mathbf{A}^I(f_{x_i})$

In [ ]:
from QQuantLib.DL.encoding_protocols import Encoding

In [ ]:
encoding_object = Encoding(
    array_function=f_norm_x, 
    array_probability=None, 
    encoding=2
)
encoding_object.run()

In [ ]:
operator_A = encoding_object.oracle
%qatdisplay operator_A --svg

In [ ]:
# It must be $|\Psi_0\rangle = |0\rangle \otimes |0\rangle_n$
encoding_object.target

In [ ]:
# all the qubits affected by the operator
encoding_object.index

#### 2.4.1. Operator $\mathbf{U}_f$

The unitary operator $\mathbf{A}^I(f_{x_i})$ needs the capability of built a unitary operator $\mathbf{U}_f$ such that: $$\mathbf{U}_f \left(|0\rangle \otimes |i\rangle_n\right) =  \big(f^{norm}_{x_i}|0\rangle +  \beta_i |1\rangle \big) \otimes |i\rangle_n$$

The operator $\mathbf{U}_f$ can be built using following procedure:

1. Given the array $f^{norm}_{x_i}$ we need to compute: $\phi_{x_i} = \arccos({f^{norm}_{x_i}})$.
2. for a given state $|i\rangle_n \otimes |0\rangle$ we need to implement a rotation around the *y-axis* over the last qubit ($|0\rangle$) controlled by the state $|i\rangle_n$ of $2*\phi_{x_i}$. So we need to build the following operation:

$$|i\rangle_n \otimes |0\rangle \rightarrow  |i\rangle_n \otimes \mathbf{R}_y(2*\phi_{x_i})|0\rangle = |i\rangle_n \otimes \left( \cos(\phi_{x_i})|0\rangle + \sin(\phi_{x_i})|1\rangle \right)  $$

3. Now undoing the $\phi_{x_i}$ and doing $\beta_i = \sin(\phi_{x_i})$ we can obtain the desired operator $\mathbf{U}_f$:

$$|i\rangle_n \otimes \left(f^{norm}_{x_i} |0\rangle +  \beta_i|1\rangle \right) = \mathbf{U}_f |i\rangle_n \otimes |0\rangle$$

The *Encoding* class from the *QQuantLib/DL/encoding_protocols* module allows us to build directly the operator $\mathbf{A}^I(f_{x_i})$ and of course the mandatory operator $\mathbf{U}_f$. This operator can be accessed using the attribute *funcion_gate* of the class:

In [ ]:
# Operator U_f
operator_Uf = encoding_object.function_gate
%qatdisplay operator_Uf --depth --svg

### 2.5 Amplitude Estimation Algorithm

Now we have the recipe for building the mandatory unitary operator $\mathbf{A}^I(f_{x_i})$. Now we need an algorithm that allows us to estimate the desired value of $\tilde{a}$ $(12)$. The **TNBS** rules of the case allow us to use any **AE** algorithm that we want. In the present implementation, we can use several different algorithms: **CQPEAE**, **IQPEAE**, **MLAE**, **IQAE** or **RQAE** (see notebook *01_AmplitudeEstimationKernel.ipynb* for more information about acronyms). 

All these algorithms have as a base operator the Grover operator of $\mathbf{A}^I(f_{x_i})$: $\mathbf{G}(\mathbf{A(f_{x_i})})$. So the first thing we need to build is this Grover operator. 



#### 2.5.1. Grover-like operator of $\mathbf{A}$

For each one of the 2 $\mathbf{A(f_{x_i})}$ operators the Grover-like operator should be constructed using $(15)$:

$$\mathbf{G}(\mathbf{A(f_{x_i})}) = \mathbf{A(f_{x_i})} \left(\hat{I} - 2|0\rangle\langle 0|\right) \mathbf{A(f_{x_i})}^{\dagger}\left(\hat{I} - 2|\Psi_0\rangle\langle \Psi_0|\right) \tag{15}$$

In the case of our **QQuantLib** the Grover-like operator building can be done in an easy way using the **Grover** function from *QQuantLib/AA/amplitude_amplification* module. The input of this function will be the following attributes of the encoding class:

* *oracle*: myqlm gate with the unitary operator $\mathbf{A(f_{x_i})}$
* *target*: the state $|\Psi_0\rangle$ of the unitary operator $\mathbf{A(f_{x_i})}$
* *index*: index affected by the unitary operator $\mathbf{A(f_{x_i})}$

So we need to provide the corresponding attributes of the object from the *Encoding* class to the **Grover** function for getting the desired **Grover** operator.

In [ ]:
from QQuantLib.AA.amplitude_amplification import grover

In [ ]:
operator_G = grover(
    oracle = encoding_object.oracle,
    target = encoding_object.target,
    index = encoding_object.index
)

In [ ]:
%qatdisplay operator_G --depth 2 --svg

### 2.5.2 Amplitude Estimation Algorithm

For each of the 2 $\mathbf{A(f_{x_i})}$ operators and their correspondent Grover-like operators $\mathbf{G}(\mathbf{A(f_{x_i})})$ the desired *Amplitude Estimation* algorithm should be used. The Python class **AE** from the **QQuantLib/AE/ae_class** module will be used for solving the **AE** problem. When this class is instantiated the following arguments should be provided:

* *oracle*: *myqlm* gate with the unitary operator $\mathbf{A(f_{x_i})}$
* *target*: the state $|\Psi_0\rangle$ of the unitary operator $\mathbf{A(f_{x_i})}$
* *index*: index affected by the unitary operator $\mathbf{A(f_{x_i})}$
* *ae_dictionary*: Python dictionary with the complete configuration of the **AE** algorithm. The most important key is the *ae_type* key where the **AE** algorithm should be specified. The following strings can be provided: **CQPEAE**, **IQPEAE**, **MLAE**, **IQAE**,**RQAE** and **MCAE**.

**NOTE**
Some descriptions of the different *AE* algorithms implemented and their configuration settings are provided in the notebook: *04_AmpliutdeEstimationAlgorithms*.

The class directly builds the *Grover* operator mandatory for the **AE** algorithm. The **run** method of the class executes the **AE** algorithm. The *AE* class have the following important attributes:

* ae_pdf: pandas DataFrame with the estimation $\tilde{a}$. It has the following columns:
    * ae: estimation of $\tilde{a}$
    * ae_l: lower bound for $\tilde{a}$ estimation (depending on the AE algorithm used)
    * ae_u: upper bound for $\tilde{a}$ estimation (depending on the AE algorithm used)
    
**BE AWARE**

The **AE** class needs a **QPU** for simulating the generated quantum circuits. The **QPU** should be provided to the *qpu* key.

In [ ]:
from QQuantLib.AE.ae_class import AE

In [ ]:
#AE base configuration dictionary (this is all the keys that the class AE can managed)
ae_dictionary = {
    #QPU
    'qpu': qpu,
    
    #Multi controlled decomposition
    'mcz_qlm': False, 
    
    #shots
    'shots': None,
    
    #MLAE
    'schedule': [
        [],
        []
    ],
    'delta' : None,
    'ns' : None,
    
    #CQPEAE
    'auxiliar_qbits_number': None,
    #IQPEAE
    'cbits_number': None,
    #IQAE & RQAQE
    'epsilon': None,
    #IQAE
    'alpha': None,
    #RQAE
    'gamma': None,
    'q': None
}

##### CQPEAE

If the Classical Quantum Phase Estimation for Amplitude Estimation (**CQPEAE**) algorithm want to be used the number os auxiliary qubits (*auxiliar_qbits_number* key) and the shots (*shots* key) should be provided in the Python configuration dictionary.


In [ ]:
ae_dictionary.update({
    'ae_type': 'CQPEAE',
    'auxiliar_qbits_number' : 10,
    'shots': 1000
})

In [ ]:
ae_object = AE(
    oracle=encoding_object.oracle,
    target=encoding_object.target,
    index=encoding_object.index,
    **ae_dictionary
)
# We need to execute run method for solving the AE problem
ae_object.run()
# Estimation of the a
cqpeae_pdf = ae_object.ae_pdf

In [ ]:
print(cqpeae_pdf)

##### IQAE

For the Iterative Quantum Amplitude Estimation Algorithm (**IQAE**) the mandatory inputs are:

* $\epsilon$ (*epsilon* key): desired semidifference of the bounds limits of the estimator returned by the algorithm.
* $\alpha$ (*alpha* key): desired probability fail of the algorithm
* shots (*shots* key): shots of each generated quantum circuit will be measured.

In [ ]:
ae_dictionary.update({
    'ae_type': 'IQAE',
    'epsilon' : 0.001,
    'alpha': 0.05,
    'shots': 1000
})
ae_object = AE(
    oracle=encoding_object.oracle,
    target=encoding_object.target,
    index=encoding_object.index,
    **ae_dictionary
)
# We need to execute run method for solving the AE problem
ae_object.run()
iqae_pdf = ae_object.ae_pdf

In [ ]:
print(iqae_pdf)

For the Maximum Likelihood Amplitude Estimation (**MLAE**) the mandatory inputs are:

* An schedule (*schedule* key): that relates the number of applications of Grover operators and the corresponding number of shots the resulting circuit will be measured.
* $\delta$ (*delta* key): parameter delta for configuring the scipy brute-force optimization algorithm used by default in our **MLAE** implementation.
* Number of grid points (*ns* key) needed by the scipy brute-force optimization algorithm used by default in our **MLAE** implementation

In [ ]:
schedule = [1, 2, 3, 4, 5]
shots = [100] * len(schedule)
ae_dictionary.update({
    'ae_type': 'MLAE',
    'schedule' : [schedule, shots],
    'delta': 1.0e-8,
    'ns': 100000
})
ae_object = AE(
    oracle=encoding_object.oracle,
    target=encoding_object.target,
    index=encoding_object.index,
    **ae_dictionary
)
# We need to execute run method for solving the AE problem
ae_object.run()
# The MLAE algorithm does not provide upper and lower bounds
mlae_pdf = ae_object.ae_pdf

In [ ]:
print(mlae_pdf)

### 2.6 Computing the integral

**AE** algorithms, usually, return the probability of getting the state $|\Psi_0\rangle$, so post-processing, for getting the estimator of the integral, should be done, as explained in (14):

$$\tilde{S}_{[a,b]} = \frac{\max(f_{x_i}) (b-a)}{2^n} \left(2^{n} \sqrt{\mathbf{P}[|\Psi_0\rangle]}\right) \tag{14}$$


In case the algorithm returns some different value an additional post-processing should be done to get the $\tilde{S}_{[a,b]}$ integral estimator.


In [ ]:
#We need to post-procces the return in order to get the correct estimator
S_cqpeae = (b-a)*normalization*(2**n*np.sqrt(cqpeae_pdf))/2**n
S_iqae = (b-a)*normalization*(2**n*np.sqrt(iqae_pdf))/2**n
S_mlae = (b-a)*normalization*(2**n*np.sqrt(mlae_pdf))/2**n

In [ ]:
print('Integral in first domain: {}'.format(sin_integral(a,b)))
print("Estimated integral using CQPEAE: {}".format(S_cqpeae["ae"][0]))
print("Estimated integral using IQAE: {}".format(S_iqae["ae"][0]))
print("Estimated integral using MLAE: {}".format(S_mlae["ae"][0]))

In [ ]:
print("Is it the integral between the bound limits of the estimator for CQPEAE algorithm: {}".format(
    (sin_integral(a,b) < S_cqpeae['ae_u'])[0] & (sin_integral(a,b) > S_cqpeae['ae_l'])[0]
))
print("Is it the integral between the bound limits of the estimator for IQAE algorithm: {}".format(
    (sin_integral(a,b) < S_iqae['ae_u'])[0] & (sin_integral(a,b) > S_iqae['ae_l'])[0]
))

#### 2.6.1 RQAE Algorithm

For the **Real Quantum Amplitude Estimation (RQAE)** algorithm the returned value is directly the **Amplitude** and not the probability (like in the other methods implemented in the library) so an additional post-process is mandatory. In this case the equation $(14)$ should be implemented as $(14.b)$

$$\tilde{S}_{[a,b]} = \frac{\max(f_{x_i}) (b-a)}{2^n} \left(2^{n} \mathbf{P}[|\Psi_0\rangle]\right) \tag{14.b}$$


The mandatory inputs of the **RQAE** are:

* $\epsilon$ (*epsilon* key): desired semidifference of the bounds limits of the estimator returned by the algorithm.
* $\gamma$ (*gamma* key): desired probability fail of the algorithm
* Amplification ratio (*q* key) between steps of the algorithm (affects to the number of applications of the Grover operator).

In [ ]:
ae_dictionary.update({
    'ae_type': 'RQAE',
    'epsilon' : 0.001,
    'alpha': None,
    'gamma': 0.05,
    'q': 2.0
})
ae_object = AE(
    oracle=encoding_object.oracle,
    target=encoding_object.target,
    index=encoding_object.index,
    **ae_dictionary
)
ae_object.run()
rqae_pdf = ae_object.ae_pdf
#We need to post-procces the return in order to get the correct estimator.
#In the RQAE case the amplitude instead of the probability is returned!!!
S_rqae = (b-a)*normalization*2**n*rqae_pdf/2**n

In [ ]:
print(S_rqae)

In [ ]:
print('Integral in first domain: {}'.format(sin_integral(a,b)))
print("Estimated integral using CQPEAE: {}".format(S_rqae["ae"][0]))
print("Is it the integral between the bound limits of the estimator: {}".format(
    (sin_integral(a,b) < S_rqae['ae_u'])[0] & (sin_integral(a,b) > S_rqae['ae_l'])[0]
))

### 2.7.Amplitude Estimation integration (q_solve_integral function).

In the other subsections of section 2, the complete workflow for **AE BTC** was described in detail. Additionally, it was explained which part of the library deals with each step of the workflow. 

The *q_solve_integral* function from *QQuantLib/finance/quantum_integration* gathers all these parts of the library for solving in a transparent way the proposed integrals using **AE** algorithms dealing with the post-processing part for giving the integral estimation. The *q_solve_integral* function was developed in the framework of the **WP 5** of the **NEASQC** project for the computation of expected values using **AE** algorithms. Depending on the inputs, this function can be used for computing the bare integrals mandatory for the **BTC** implementation. The input is a Python dictionary and for solving bare integrals following inputs should be provided:

* *array_function*: numpy array with the desired array function for Riemann sum
* *array_probability*:  numpy array with the probability distribution for computation of expected values. In our case, this will be None.
* *encoding*: int for selecting the encoding. In our case, It will be 2.

The other keys of the input dictionary are for selecting and configuring the **AE** algorithm (and are the same as for **AE** class).

The outputs of the *q_solve_integral* function are:
* ae_estimation: pandas DataFrame with the desired integral computation and the upper and lower limits if applied.
* solver_ae: object based on the AE class

The *q_solve_integral* returns directly the integral of the input function directly. So the returned will be: $2^{n} \sqrt{\mathbf{P}[|\Psi_0\rangle]}$ (or the $2^{n} \mathbf{P}[|\Psi_0\rangle]$ for the **RQAE** algorithm)


**BE AWARE!!**

This is why we kept the  $2^n$ in the integral computation, for taking advance of the implemented integral in the *q_solve_integral* function!!

$$
\tilde{S}_{[a,b]} =   \left\{
\begin{array}{ll}
      \frac{\max(f_{x_i}) (b-a)}{2^n} \left(2^{n} \sqrt{\mathbf{P}[|\Psi_0\rangle]}\right) & For \; probability \; measurements \\
      \frac{\max(f_{x_i}) (b-a)}{2^n} \left(2^{n} \mathbf{P}[|\Psi_0\rangle]\right) & For \; amplitude \; measurements \\
\end{array} 
\right.
$$

In [ ]:
from QQuantLib.finance.quantum_integration import q_solve_integral

In [ ]:
#AE base configuration dictionary
ae_dictionary = {
    #QPU
    'qpu': qpu,
    #Multi controlled decomposition
    'mcz_qlm': False, 
    
    #shots
    'shots': None,
    
    #MLAE
    'schedule': [
        [],
        []
    ],
    'delta' : None,
    'ns' : None,
    
    #CQPEAE
    'auxiliar_qbits_number': None,
    #IQPEAE
    'cbits_number': None,
    #IQAE & RQAQE
    'epsilon': None,
    #IQAE
    'alpha': None,
    #RQAE
    'gamma': None,
    'q': None
}
#IQAE configuration
ae_dictionary.update({
    'ae_type': 'IQAE',
    'epsilon' : 0.001,
    'alpha': 0.05,
    'shots':100
})

encoding_dict = {
    "array_function" : f_norm_x,
    "array_probability" : None,
    "encoding" : 2    
}

ae_dictionary.update(encoding_dict)

In [ ]:
solution, solver_object = q_solve_integral(**ae_dictionary)
#Integral of the input array is returned so only normalization has to be taken into account
S_iqae = normalization*solution*(b-a)/2**n

In [ ]:
S_iqae

In [ ]:
print("Is it the integral between the bound limits of the estimator: {}".format(
    (sin_integral(a,b) < S_iqae['ae_u'])[0] & (sin_integral(a,b) > S_iqae['ae_l'])[0]
))

When using the **RQAE** algorithm *q_solve_integral* deals with the internal normalisations and the integral of the input array is returned in an transparent way.

In [ ]:
#AE base configuration dictionary
ae_dictionary = {
    #QPU
    'qpu': qpu,
    #Multi controlled decomposition
    'mcz_qlm': False, 
    
    #shots
    'shots': None,
    
    #MLAE
    'schedule': [
        [],
        []
    ],
    'delta' : None,
    'ns' : None,
    
    #CQPEAE
    'auxiliar_qbits_number': None,
    #IQPEAE
    'cbits_number': None,
    #IQAE & RQAQE
    'epsilon': None,
    #IQAE
    'alpha': None,
    #RQAE
    'gamma': None,
    'q': None
}
#IQAE configuration
ae_dictionary.update({
    'ae_type': 'RQAE',
    'epsilon' : 0.001,
    'gamma': 0.05,
    'q' : 1.2
})

encoding_dict = {
    "array_function" : f_norm_x,
    "array_probability" : None,
    "encoding" : 2    
}

ae_dictionary.update(encoding_dict)

In [ ]:
solution, solver_object = q_solve_integral(**ae_dictionary)
#Integral of the input array is returned so only normalization has to be taken into account
S_rqae = normalization*solution*(b-a)/2**n

In [ ]:
S_rqae

In [ ]:
print("Is it the integral between the bound limits of the estimator: {}".format(
    (sin_integral(a,b) < S_rqae['ae_u'])[0] & (sin_integral(a,b) > S_rqae['ae_l'])[0]
))

## 3. Getting the metrics

Once the amplitude estimator of the integral, $\tilde{S}$, is obtained following metrics should be  computed:

* *Integral Absolute error* between the ae estimator and the Riemann sum of the integral to compute $(1)$: $\epsilon = |\tilde{S} - S_{[a,b]}|$

* *Oracle calls*: total number of calls of the operator $\mathbf{A}$ (the number of shots should be taken into account in this calculation).

* *Elapsed time*:The complete time for getting $\tilde{S}$. This time will include all the complete steps this is (it is not necessary to have times for each step):
    * Discretization time
    * Creation of operator $\mathbf{A}$
    * Creation of the Grover-like operator: $\mathbf{G}$
    * Execution of the *amplitude Estimation* algorithm
    * Post-processing execution
    * Metrics calculation.

## 4. AE integration Workflow summary

We have all the pieces needed to do a complete integration using **AE** algorithms. Now we summarize the procedure:

1. Execute the following steps for each one of the 2 integration described intervals in Section 1
    1. Execute the following steps from n = 4 to the maximum number of qubits. For each $n$ execute the following steps:
        1. Create the domain discretization (sub-section 2.1)
        2. Create the array with the corresponding sine function discretization (sub-section 2.2)
        3. Compute normalization of the array (sub-section 2.3)
        4. Create the $\mathbf{A}$ oracle operator for encoding the array (see sub-section 2.4)
        5. Create the corresponding Grover-like operator, $\mathbf{G}(\mathbf{A(f_{x_i})})$, from  $\mathbf{A}$ (see section 2.5.1)
        6. Execute the tested *amplitude estimation* algorithm properly configured using the operators $\mathbf{G}(\mathbf{A(f_{x_i})})$, and $\mathbf{A}$ (see section 2.5.2)
        7. With the *amplitude estimation* algorithm result compute the integral (see section 2.6)
        8. Compute the desired metrics (see section 3).

For doing this complete workflow the *sine_integral* function from **QQuantLib/ae_sine_integral** was implemented. The inputs for this function are:

* *n_qbits*: number of qubits for interval discretization.
* *interval*: int for selecting the integration interval.
* *ae_dictionary*: python dictionary with the complete configuration of the *Amplitude Estimation* algorithm.

The return is a pandas DataFrames with the complete information of the computation.

In [ ]:
#AE base configuration dictionary
ae_dictionary = {
    #QPU
    'qpu': qpu,
    #Multi controlled decomposition
    'mcz_qlm': False, 
    
    #shots
    'shots': None,
    
    #MLAE
    'schedule': [
        [],
        []
    ],
    'delta' : None,
    'ns' : None,
    
    #CQPEAE
    'auxiliar_qbits_number': None,
    #IQPEAE
    'cbits_number': None,
    #IQAE & RQAQE
    'epsilon': None,
    #IQAE
    'alpha': None,
    #RQAE
    'gamma': None,
    'q': None
}
#IQAE configuration
ae_dictionary.update({
    'ae_type': 'IQAE',
    'epsilon' : 0.001,
    'alpha': 0.05,
    'shots': 100
})

In [ ]:
from QQuantLib.ae_sine_integral import sine_integral

### Integration for the first interval 

In [ ]:
#first interval integral benchmarking
pdf = sine_integral(4, 0, ae_dictionary)

In [ ]:
pdf.columns

In [ ]:
pdf[['integral_ae', 'integral_ae_l', 'integral_ae_u', 'exact_integral', 'IntegralAbsoluteError']]

### Integration for the second interval 

In [ ]:
#second integral is negative so the any method except RQAE will fail
pdf = sine_integral(4, 1, ae_dictionary)

In [ ]:
pdf[['integral_ae', 'integral_ae_l', 'integral_ae_u', 'exact_integral', 'IntegralAbsoluteError']]

In [ ]:
# Only RQAE can deal with negative integrals
#RQAE configuration
ae_dictionary.update({
    'ae_type': 'RQAE',
    'epsilon' : 0.001,
    'gamma': 0.05,
    'q' : 2.0,
    'alpha': None
})
#second integral is negative
pdf = sine_integral(4, 1, ae_dictionary)

In [ ]:
pdf[['integral_ae', 'integral_ae_l', 'integral_ae_u', 'exact_integral', 'IntegralAbsoluteError']]

## 5. Command line execution and JSON configuration files.

The  **BTC_02_AE/ae_sine_integral** can be executed from the command line to run a complete integration using **AE** algorithms. A help of the input of the program can be obtained by using:

    python ae_sine_integral.py -h


    usage: ae_sine_integral.py [-h] [-n_qbits N_QBITS] [-interval INTERVAL] [-repetitions REPETITIONS]
                               [-id ID] [-json_ae JSON_AE] [-json_qpu JSON_QPU] [-folder FOLDER_PATH]
                               [--count] [--print] [--save] [--exe]

    optional arguments:
      -h, --help            show this help message and exit
      -n_qbits N_QBITS      Number of qbits for interval discretization.
      -interval INTERVAL    Integration Interval. valid Values 0 or 1
      -repetitions REPETITIONS
                            Number of repetitions the integral will be computed.Default: 1
      -id ID                For executing only one element of the list
      -json_ae JSON_AE      JSON AE algorithm configuration
      -json_qpu JSON_QPU    JSON with the qpu configuration
      -folder FOLDER_PATH   Path for storing folder
      --count               For counting elements on the list
      --print               For printing the AE algorihtm configuration.
      --save                For saving results
      --exe                 For executing program


The **AE** configuration should be provided as JSON file to the **-json_ae** (several examples can be found under the folder **BTC_02_AE/jsons**). 

The **QPU** configuration should be provided as JSON file to the **-json_qpu** (several examples can be found under the folder **QQuantLib/qpu/**). 

The  *JSON* files can be configured for generating a lot of possible configurations. To know the final number of configurations the **--count** can be provided.

Example: if the JSON files were not modified the following command:

    python ae_sine_integral.py  -n_qbits 6  -interval 0 -json_ae ../jsons/integral_iqae_configuration.json -json_qpu qpu/qpu_ideal.json --count
    
will provide a **12**. So until 12 different configurations can be selected (all the possible combinations fo the *AE* and *QPU* configurations).

For selecting one possible configuration the **-id ID** argument should be used. The **--print** argument can be used combined with **-id ID** for showing the ID configuration.

Example: if the JSON files were not modified the following command:

    python ae_sine_integral.py  -n_qbits 6  -interval 0 -json_ae ../jsons/integral_iqae_configuration.json -json_qpu qpu/qpu_ideal.json -id 2 --print
    
will provide the 2nd configuration:

    {'ae_type': 'IQAE', 'schedule': None, 'delta': None, 'ns': None, 'auxiliar_qbits_number': None, 'cbits_number': None, 'epsilon': 0.001, 'alpha': 0.05, 'gamma': None, 'q': None, 'multiplexor': True, 'mcz_qlm': False, 'shots': 1000, 'qpu_type': 'linalg', 't_gate_1qb': None, 't_gate_2qbs': None, 't_readout': None, 'depol_channel': {'active': False, 'error_gate_1qb': None, 'error_gate_2qbs': None}, 'idle': {'amplitude_damping': False, 'dephasing_channel': False, 't1': None, 't2': None}, 'meas': {'active': False, 'readout_error': None}}
    
Finally, the **--exe** method in combination with the **-id ID** will execute the complete **AE** integration using the *ID* configuration.

Example: if the JSON files were not modified the following command:

    python ae_sine_integral.py  -n_qbits 6  -interval 0 -json_ae ../jsons/integral_iqae_configuration.json -json_qpu qpu/qpu_ideal.json -id 2 --exe
    
will execute the **AE** integration corresponding to the 2nd configuration and the obtained pandas DF will be printed.

The **--save** arguments allow the user to save the results in a CSV and the **-folder FOLDER** allows the user to provide a folder for storing the obtained files.

### Complete execution example:

     python ae_sine_integral.py  -n_qbits 6  -interval 0 -json_ae ../jsons/integral_iqae_configuration.json -json_qpu qpu/qpu_ideal.json -id 2 --exe --save -folder ./test/

The before command will compute the integral of the sine of the interval 0, for a domain discretization of 6 qubits. The configuration of the algorithm is selected using the JSON file **../jsons/integral_iqae_configuration.json** and the configuration for the **QPU** will be obtained from the JSON file **qpu/qpu_ideal.json**. Of all the possible combinations **AE** algorithm and **QPU** configurations the second one will be executed. The CSV will be saved in the **./test/** folder